# 🧹 Data Preprocessing & Feature Engineering
## Student Dropout Prediction Project

**Goal:** Prepare the merged dataset for machine learning by handling missing values, encoding features, scaling, and balancing classes.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import sys
import os
import importlib
import warnings
import joblib
warnings.filterwarnings('ignore')

# Add parent directory to path
sys.path.append('..')
import config
importlib.reload(config)

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 1. Load Merged Data

In [2]:
try:
    df = pd.read_csv(config.MERGED_DATASET_PATH)
    print(f"✓ Dataset loaded: {df.shape[0]:,} rows, {df.shape[1]} columns")
except FileNotFoundError:
    print("❌ Error: Merged dataset not found. Run 00_merge_datasets_v2.ipynb first.")

✓ Dataset loaded: 9,013 rows, 17 columns


## 2. Handle Missing Values

In [3]:
# Separate features and target
X = df.drop(columns=['Target', 'Dataset_Source'])
y = df['Target']

# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

print(f"Numerical Features: {len(numerical_cols)}")
print(f"Categorical Features: {len(categorical_cols)}")

# Define Imputers
# Numerical -> Median (robust to outliers)
num_imputer = SimpleImputer(strategy='median')

# Categorical -> Most Frequent (Mode)
cat_imputer = SimpleImputer(strategy='most_frequent')

# Apply Imputation (only if columns exist)
if len(numerical_cols) > 0:
    X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])
    
if len(categorical_cols) > 0:
    X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

print("✓ Missing values imputed")
print(f"Remaining missing: {X.isnull().sum().sum()}")

Numerical Features: 15
Categorical Features: 0
✓ Missing values imputed
Remaining missing: 0


## 3. Feature Encoding & Scaling

In [4]:
# Encode Target (Multi-class)
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y)

print(f"Target Classes: {target_encoder.classes_}")

# Create Preprocessor (ColumnTransformer)
transformers = []

# Add numerical transformer
if len(numerical_cols) > 0:
    transformers.append(('num', Pipeline([
        ('scaler', StandardScaler())
    ]), numerical_cols))

# Add categorical transformer (only if categorical columns exist)
if len(categorical_cols) > 0:
    transformers.append(('cat', Pipeline([
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]), categorical_cols))

preprocessor = ColumnTransformer(transformers=transformers)

# Fit and transform
X_processed = preprocessor.fit_transform(X)

# Get feature names after preprocessing
feature_names = []
if len(numerical_cols) > 0:
    feature_names.extend(numerical_cols.tolist())
if len(categorical_cols) > 0:
    feature_names.extend(preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_cols).tolist())

X_processed_df = pd.DataFrame(X_processed, columns=feature_names)

print(f"✓ Data processed. New shape: {X_processed_df.shape}")

Target Classes: ['Dropout' 'Enrolled' 'Graduate']
✓ Data processed. New shape: (9013, 15)


## 4. Train-Test Split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X_processed_df,
    y_encoded,
    test_size=config.TEST_SIZE,
    random_state=config.RANDOM_STATE,
    stratify=y_encoded
)

print(f"✓ Split complete")
print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Class Distribution (Train): {np.bincount(y_train)}")
print(f"Class Distribution (Test):  {np.bincount(y_test)}")

✓ Split complete
Train: (7210, 15), Test: (1803, 15)
Class Distribution (Train): [2351 1265 3594]
Class Distribution (Test):  [588 316 899]


## 5. Handle Class Imbalance (SMOTE)

In [6]:
smote = SMOTE(random_state=config.RANDOM_STATE)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"✓ SMOTE applied")
print(f"Original Train: {X_train.shape}")
print(f"Balanced Train: {X_train_balanced.shape}")
print(f"Balanced Class Distribution: {np.bincount(y_train_balanced)}")

✓ SMOTE applied
Original Train: (7210, 15)
Balanced Train: (10782, 15)
Balanced Class Distribution: [3594 3594 3594]


## 6. Save Processed Data

In [7]:
# Convert back to DataFrame
train_df = pd.DataFrame(X_train_balanced, columns=feature_names)
train_df['Target'] = y_train_balanced

test_df = pd.DataFrame(X_test, columns=feature_names)
test_df['Target'] = y_test

# Save
train_df.to_csv(config.TRAIN_DATA_PATH, index=False)
test_df.to_csv(config.TEST_DATA_PATH, index=False)

# Save Preprocessor & Encoder
joblib.dump(preprocessor, config.MODEL_DIR / "preprocessor.pkl")
joblib.dump(target_encoder, config.MODEL_DIR / "target_encoder.pkl")

print(f"\n✓ Processed data saved:")
print(f"  Train: {config.TRAIN_DATA_PATH}")
print(f"  Test:  {config.TEST_DATA_PATH}")
print(f"✓ Preprocessor saved: {config.MODEL_DIR / 'preprocessor.pkl'}")
print(f"✓ Target Encoder saved: {config.MODEL_DIR / 'target_encoder.pkl'}")


✓ Processed data saved:
  Train: D:\Santosh_minor\notebooks\..\data\processed\train_data.csv
  Test:  D:\Santosh_minor\notebooks\..\data\processed\test_data.csv
✓ Preprocessor saved: D:\Santosh_minor\notebooks\..\models\preprocessor.pkl
✓ Target Encoder saved: D:\Santosh_minor\notebooks\..\models\target_encoder.pkl
